# 測試多模態模型提取 PDF 中需要欄位：


In [1]:
# 處理 Text to JSON
import json
import re
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

load_dotenv()
project_id = "nthurc-aisearch-202406"
model_name = "gemini-1.5-flash-001"


def parse_json_from_text(response_text):
    """
    從包含 JSON 或 JSON Lines 的文字中解析 JSON 資料。

    Args:
        response_text: 包含 JSON 或 JSON Lines 資料的字串。

    Returns:
        一個包含解析後的 JSON 資料的列表。
    """

    # 使用正則表達式去除開頭的非 JSON 字元
    json_strings = re.sub(
        r'^[^\{]+', '', response_text).rstrip('`').strip().split('\n')

    data = []
    for json_string in json_strings:
        # 檢查是否為空字串，若是則跳過
        if not json_string.strip():
            continue

        try:
            data.append(json.loads(json_string))
        except json.JSONDecodeError as e:
            print(json_string)
            print(f"Error decoding JSON: {e}")

    return data


def process_taxpayer_document(project_id, model_name, part, prompt):
    vertexai.init(project=project_id, location="asia-east1")
    model = GenerativeModel(model_name)

    contents = [part, prompt]
    response = model.generate_content(contents)

    print(response.text)
    # print("=============")
    # print("處理過後的 JSON : \n\n", parse_json_from_text(response.text))

    return response.text

### 1. 所得相關資料提取

- 目前使用 Vertex AI ( Gemini-1.5-flash 多模態模型 ) 來提取 PDF 中的資料
- 明確定義我們需要的提取的內容
- 定義 JSON
- 設計 Prompt 並讓 Gemini 理解任務
- 去得輸出，並改成可解析的結構


#### 這部分目前需要釐清的問題 :

- 電子財產：這部分的檔案是否也要在這一塊進行處理？
- 這一部分的檔案除了目前欄位以外是否還有其他欄位有需求


In [3]:
json_format = '''{
  "taxYear": 提取此所得資料清單[年度]資訊，位於文檔頂部,
  "IssuingAuthority": 提取此所得稅[核發單位]文字資訊放於此欄位,
  "taxpayerName": 提取[所得人姓名],
  "taxpayerID": 提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼,
  "reportDate": 提取此報告[查調日期]，位於文檔右上角,
  "totalIncome": 提取此所得人總所得，位於文檔下方有一個[給付總額合計]的資訊,
}'''

prompt = f"""
You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

taxpay_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/所得/所得14.pdf"
taxpay_pdf_file = Part.from_uri(
    taxpay_pdf_file_uri, mime_type="application/pdf")
json = process_taxpayer_document(
    project_id, model_name, taxpay_pdf_file, prompt)
print(json)

json_strings = re.sub(
    r'^[^\{]+', '', json).rstrip('`').strip().split('\n')

```json
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃德旺", "taxpayerID": "R124075657", "reportDate": "111年07月11日", "totalIncome": "724,894"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃春萬", "taxpayerID": "R120335830", "reportDate": "111年07月11日", "totalIncome": "107"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃麗珍", "taxpayerID": "Q221609858", "reportDate": "111年07月11日", "totalIncome": "0"}
```
處理過後的 JSON : 

 [{'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃德旺', 'taxpayerID': 'R124075657', 'reportDate': '111年07月11日', 'totalIncome': '724,894'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃春萬', 'taxpayerID': 'R120335830', 'reportDate': '111年07月11日', 'totalIncome': '107'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃麗珍', 'taxpayerID': 'Q221609858', 'reportDate': '111年07月11日', 'totalIncome': '0'}]
('```json\n{"tax

# 所得文檔正式處理

- 處理 30 個所得文檔，提取其中 Jsonl 資料，並合併生成一個新的 Jsonl 檔案
- 再將 JsonL 資料轉換成 Dataframe
- 最後將 Dataframe 儲存成 CSV 檔案


In [8]:
import json
import re
import vertexai
from dotenv import load_dotenv
from google.cloud import storage
from vertexai.generative_models import GenerativeModel, Part
import pandas as pd
import logging

# 設置日誌
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


def process_taxpayer_document(project_id, model_name, part, prompt):
    """
    处理纳税人文档的函数。

    参数：
    - project_id：项目ID，用于初始化Vertex AI。
    - model_name：生成模型的名称。
    - part：文档的一部分。
    - prompt：生成内容的提示。

    返回值：
    - response.text：生成的内容。

    异常：
    - 如果处理文档时发生错误，将记录错误并返回None。
    """
    try:
        # 初始化Vertex AI
        vertexai.init(project=project_id, location="asia-east1")

        # 创建生成模型对象
        model = GenerativeModel(model_name)

        # 构建内容列表
        contents = [part, prompt]

        # 生成内容
        response = model.generate_content(contents)

        # 返回生成的内容
        return response.text
    except Exception as e:
        # 记录错误并返回None
        logging.error(f"处理文档时发生错误：{e}")
        return None


def extract_json_strings(json_response):
    """
    提取JSON响应中的字符串函数。

    参数：
    - json_response：JSON响应字符串。

    返回值：
    - 字符串列表：提取的JSON响应中的字符串列表。

    说明：
    - 如果json_response为None，则返回空列表。
    - 移除json_response中的所有```json或```jsonl标记。
    - 使用正则表达式找出所有的JSON对象。
    - 解析每个JSON对象并重新序列化，以确保它们是有效的JSON。
    - 返回有效的JSON字符串列表。

    注意：
    - 忽略无效的JSON对象，继续处理下一个对象。
    """
    if json_response is None:
        return []

    # 移除所有的 ```json 或 ```jsonl 標記
    cleaned_response = re.sub(r'```(?:json|jsonl)?\s*', '', json_response)

    # 使用正則表達式找出所有的 JSON 對象
    json_objects = re.findall(r'\{[^{}]*\}', cleaned_response)

    # 解析每個 JSON 對象並重新序列化，以確保它們是有效的 JSON
    valid_json_strings = []
    for obj in json_objects:
        try:
            parsed = json.loads(obj)
            valid_json_strings.append(json.dumps(parsed, ensure_ascii=False))
        except json.JSONDecodeError:
            print(f"Invalid JSON: {obj}")
            continue  # 忽略無效的 JSON

    return valid_json_strings


def create_json_format(file_uri):
    """
    创建JSON格式的函数。

    参数：
    - file_uri：文件的URI。

    返回值：
    - JSON对象：包含文件URI和各个字段的JSON对象。

    字段说明：
    - gsURL：文件的Google Cloud Storage URL。
    - taxYear：提取此所得資料清單[年度]資訊，位於文檔頂部，格式為 XXX 年度。
    - IssuingAuthority：提取此所得稅[核發單位]文字資訊放於此欄位。
    - taxpayerName：提取[所得人姓名]。
    - taxpayerID：提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼。
    - reportDate：提取此報告[查調日期]，位於文檔右上角。
    - totalIncome：提取此所得人總所得，位於文檔下方有一個[給付總額合計]的資訊，格式為數字。
    """
    return {
        "gsURL": file_uri,
        "taxYear": "提取此所得資料清單[年度]資訊，位於文檔頂部",
        "IssuingAuthority": "提取此所得稅[核發單位]文字資訊放於此欄位",
        "taxpayerName": "提取[所得人姓名]",
        "taxpayerID": "提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼, 由一個大寫字母(注意 I 不要認為是 1) + 9 位數字組成",
        "reportDate": "提取此報告[查調日期]，位於文檔右上角",
        "totalIncome": "提取此所得人總所得，位於文檔下方有一個[給付總額合計]的資訊"
    }


def create_prompt(json_format):
    """
    创建提示信息的函数。

    参数：
    - json_format：包含文件URI和各个字段的JSON对象。

    返回值：
    - 提示信息字符串。

    提示信息说明：
    - 提示用户作为PDF文档提取器，提供了来自台湾财政部的财产收入文档。
    - 要求用户从每一页中提取信息，并将其组织成JSONL格式返回。
    - 明确定义了需要的内容，并要求用户根据PDF中的内容提取值并返回。
    - 提供了JSON格式的示例，其中包含了各个字段的说明。
    - 要求用户仔细审查PDF文件的每一页，并提取信息以生成相应的JSON响应。
    - 值可以接受繁体中文输出。
    - 提示用户在输出之前仔细思考，并返回结论。
    - 用户的答案对我的职业非常重要。

    返回值说明：
    - 返回包含提示信息的字符串。
    """
    return f"""
    You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
    ```json format : {json.dumps(json_format, ensure_ascii=False, indent=2)}```
    Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.
    JSONL :
    """


def process_all_documents(project_id, model_name, bucket_name, file_prefix, data_type):
    """
    处理所有文档的函数。

    参数：
    - project_id：项目ID，用于初始化Vertex AI。
    - model_name：生成模型的名称。
    - bucket_name：存储桶的名称。
    - file_prefix：文件的前缀。

    返回值：
    - DataFrame：包含所有文档提取的信息的DataFrame。

    异常：
    - 如果解码JSON时发生错误，将记录错误并返回空的DataFrame。

    说明：
    - 初始化存储客户端。
    - 获取存储桶对象。
    - 初始化Vertex AI。
    - 创建空列表以存储所有的JSON字符串。
    - 获取存储桶中指定前缀的所有Blob对象。
    - 计算总文件数。
    - 遍历每个Blob对象。
        - 构建文件的URI。
        - 记录正在处理的文件信息。
        - 创建JSON格式对象。
        - 创建提示信息。
        - 从URI创建Part对象。
        - 处理纳税人文档并获取JSON响应。
        - 如果JSON响应不为空，则提取JSON字符串并添加到列表中。
        - 记录成功处理的文件信息。
        - 否则，记录处理失败的文件信息。
    - 将所有的JSON字符串写入文件。
    - 将JSON字符串转换为DataFrame并返回。

    注意：
    - 如果解码JSON时发生错误，将记录错误并返回空的DataFrame。
    """
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)
    vertexai.init(project=project_id)
    all_json_strings = []

    blobs = list(bucket.list_blobs(prefix=file_prefix))
    total_files = len(blobs)

    for index, blob in enumerate(blobs, 1):
        file_uri = f"gs://{bucket_name}/{blob.name}"
        logging.info(f"Processing file {index}/{total_files}: {blob.name}")

        json_format = create_json_format(file_uri)
        prompt = create_prompt(json_format)

        taxpay_pdf_file = Part.from_uri(file_uri, mime_type="application/pdf")
        json_response = process_taxpayer_document(
            project_id, model_name, taxpay_pdf_file, prompt)

        if json_response:
            print(json_response)
            print("\n ====== \n")
            json_strings = extract_json_strings(json_response)
            print(json_strings, "\n\n\n")
            all_json_strings.extend(json_strings)
            logging.info(f"Successfully processed file: {blob.name}")
        else:
            logging.warning(f"Failed to process file: {blob.name}")

    with open(data_type + 'combined_output.jsonl', 'w', encoding='utf-8') as f:
        for json_string in all_json_strings:
            f.write(json_string + '\n')

    try:
        data = [json.loads(js) for js in all_json_strings]
        df = pd.DataFrame(data)
        return df
    except json.JSONDecodeError as e:
        logging.error(f"Error decoding JSON: {e}")
        return pd.DataFrame()

In [ ]:
if __name__ == "__main__":
    load_dotenv()
    project_id = "nthurc-aisearch-202406"
    model_name = "gemini-1.5-flash-001"
    bucket_name = "nthurc-docai-poc"
    file_prefix = "申請資料-4要件/所得/"
    data_type = "所得"

    df = process_all_documents(
        project_id, model_name, bucket_name, file_prefix, data_type)
    print(df.head())
    print(f"Total rows in DataFrame: {len(df)}")

In [45]:
df[9:13]

,gsURL,taxYear,IssuingAuthority,taxpayerName,taxpayerID,reportDate,totalIncome
9,gs://nthurc-docai-poc/申請資料-4要件/所得/所得12.pdf,110年度,財政部北區國稅局新店稽徵所,賴彥成,1100072456,111年07月14日,"276,759"
10,gs://nthurc-docai-poc/申請資料-4要件/所得/所得12.pdf,110年度,財政部北區國稅局新店稽徵所,張倚禎,T222834750,111年07月14日,65
11,gs://nthurc-docai-poc/申請資料-4要件/所得/所得12.pdf,110年度,財政部北區國稅局新店稽徵所,賴禹昇,1100623257,111年07月14日,0
12,gs://nthurc-docai-poc/申請資料-4要件/所得/所得12.pdf,110年度,財政部北區國稅局新店稽徵所,賴竑佑,F135360168,111年07月14日,0


In [41]:
df.to_csv('./所得測試資料.csv', index=True)

## 2. 戶籍謄本相關資料提取

- 跟上一個方法一樣，通過定義需要的欄位後,讓多模態模型進行視覺的資料提取並轉換成文字回傳


#### 這部分需要釐清的部分 :

- 戶籍謄本帶現戶全戶這個後綴，是需要提取還是判斷是否有這個欄位？
- 提取這個現戶全戶的目的是什麼，為了區分戶口名簿嗎？
- 是否只需要提取戶長資訊即可？


In [4]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
        "gsURL": file_uri,
        "IssueDate": "取得此 PDF 中[列印日期/時間]中的日期部分",
        "IsFamilyRegisterExtract": "判斷此 PDF 中是否帶有(現戶全戶)在文件標題，如果有返回 True, 否則返回 False, 如果前綴是戶口名簿則輸出 None",
        "householdId": "取得此 PDF 中的[戶號]",
        "householdAddress": "取得此 PDF 中的[戶籍地址]",
        "membersOfHousehold": {
            "description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容",
            "members": [
                {
                    "name": "取得此成員的姓名",
                    "title": "取得此成員的稱謂",
                    "birthDate": "取得此成員的出生日期",
                    "PersonID": "取得此成員的身份證字號",
                    "spouse": "取得此成員的配偶姓名",
                    "father": "取得此成員的父親姓名",
                    "mother": "取得此成員的母親姓名",
                    "note": "取得此成員的[記事]內所有紀錄的資訊"
                }
            ]
        }
    }'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a household registration book or household registration transcript provided by the Taiwan government. You will extract the specified information from the PDF to JSON and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSON :
"""

FamilyRegisterExtract_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶謄8.pdf"
FamilyRegisterExtract_pdf_file = Part.from_uri(
    FamilyRegisterExtract_pdf_file_uri, mime_type="application/pdf")
contents = [FamilyRegisterExtract_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```json
{
        "gsURL": "gs://your-bucket/household_registration_book.pdf",
        "IssueDate": "111/07/07",
        "IsFamilyRegisterExtract": True,
        "householdId": "F0A03MC9",
        "householdAddress": "新北市永和區中溪里007鄰仁愛路259號五樓",
        "membersOfHousehold": {
            "description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容",
            "members": [
                {
                    "name": "黃郁羚",
                    "title": "戶長",
                    "birthDate": "民國74年6月18日",
                    "PersonID": "F225951328",
                    "spouse": "簡川翔",
                    "father": "黃世佳",
                    "mother": "吳瑞卿",
                    "note": "在本鎮和平街19號達生婦產科出生民國74年7月5日申登。原住本鎮八張里5鄰民族街81號黃金泉戶內民國85年7月13日住址變更。民國103年5月17日與簡川翔結婚。原住新北市三峽區二鬮里012鄰二鬮路136巷123號三樓吳瑞卿戶內民國103年10月31日遷入登記。男簡靖恩統一編號F132184228民國103年10月26日出生。男簡愷均國民身分證統一編號F135128633民國108年3月25日出生。"
                },
                {
                    "name": "簡靖恩",

# 戶籍文檔正式處理

- 處理 30 個戶籍以及戶口文檔，提取其中 Jsonl 資料，並合併生成一個新的 Jsonl 檔案
- 應為有嵌套，所以不能轉成 Dataframe 格式
- 再將 JsonL 資料轉換成標準格式的 json 格式
- 將 json 鋪平後轉成 CSV 格式


In [9]:
def extract_json_strings(json_response):
    return re.sub(
        r'^[^\{]+', '', json_response).rstrip('`').strip().split('\n')


def create_json_format(file_uri):
    return {
        "gsURL": file_uri,
        "IssueDate": "取得此 PDF 中[列印日期/時間]中的日期部分",
        "IsFamilyRegisterExtract": "判斷此 PDF 中是否帶有(現戶全戶)在文件標題，如果有返回 True, 否則返回 False, 如果前綴是戶口名簿則輸出 None",
        "householdId": "取得此 PDF 中的[戶號]",
        "householdAddress": "取得此 PDF 中的[戶籍地址]",
        "membersOfHousehold": {
            "description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容",
            "members": [
                {
                    "name": "取得此成員的姓名",
                    "title": "取得此成員的稱謂",
                    "birthDate": "取得此成員的出生日期",
                    "PersonID": "取得此成員的身份證字號",
                    "spouse": "取得此成員的配偶姓名",
                    "father": "取得此成員的父親姓名",
                    "mother": "取得此成員的母親姓名",
                    "note": "取得此成員的[記事]內所有紀錄的資訊"
                }
            ]
        }
    }


def create_prompt(json_format):
    return f"""
    You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
    ```json format : {json.dumps(json_format, ensure_ascii=False, indent=4)}```
    Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.
    JSONL :
    """


file_prefix = "申請資料-4要件/戶籍/"
data_type = "戶籍"

process_all_documents(
    project_id, model_name, bucket_name, file_prefix, data_type)

2024-08-02 14:28:21,301 - INFO - Processing file 1/30: 申請資料-4要件/戶籍/戶口28.pdf
2024-08-02 14:28:28,415 - INFO - Successfully processed file: 申請資料-4要件/戶籍/戶口28.pdf
2024-08-02 14:28:28,417 - INFO - Processing file 2/30: 申請資料-4要件/戶籍/戶口5.pdf


```jsonl
{"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶口28.pdf", "IssueDate": "111/06/01", "IsFamilyRegisterExtract": "True", "householdId": "F5375388", "householdAddress": "桃園市桃園區中埔里004鄰永安路919巷9號", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "包佩怡", "title": "肆女", "birthDate": "民國75年7月10日", "PersonID": "F226229325", "spouse": "鄭伸榮", "father": "包東平", "mother": "黃碧秀", "note": "在永和市光復街15號出生(民國75年7月14日申報)。民國105年7月7日申請與鄭\n伸榮結婚登記,民國105年7月10日結婚登記。女鄭嘉韵國民身分證統一編號H227586045\n民國107年4月13日出生。"}]}}
```


['{"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶口28.pdf", "IssueDate": "111/06/01", "IsFamilyRegisterExtract": "True", "householdId": "F5375388", "householdAddress": "桃園市桃園區中埔里004鄰永安路919巷9號", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "包佩怡", "title": "肆女", "birthDate": "民國75年7月10日", "PersonID": "F226229325", "spouse"

2024-08-02 14:28:40,017 - INFO - Successfully processed file: 申請資料-4要件/戶籍/戶口5.pdf
2024-08-02 14:28:40,018 - INFO - Processing file 3/30: 申請資料-4要件/戶籍/戶口7.pdf


```jsonl
{"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶口5.pdf", "IssueDate": "111/07/15", "IsFamilyRegisterExtract": None, "householdId": "FOA2RMW5", "householdAddress": "新北市板橋區國泰里005鄰忠孝路忠義巷15弄96號三樓", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "梁乙欽", "title": "戶長", "birthDate": "民國45年3月10日", "PersonID": "N120769673", "spouse": "易莉菲", "father": "梁秀森", "mother": "梁張摘", "note": "出生地:台灣省彰化縣。民國78年1月17日與梁江絲離婚。民國82年5月20日與易美蓉結婚。民國82年5月31日申登。妻梁易美蓉民國83年2月8日姓名變更民國93年8月27日申登。妻梁易修禪民國91年7月23日姓名變更民國93年8月27日申登。妻梁易華茜民國93年8月27日撤冠姓。原登記配偶姓名易華茜因配偶改名民國106年5月5日變更民國106年5月5日經彰化縣員林市戶政事務所逕為變更登記。"}, {"name": "易莉菲", "title": "妻", "birthDate": "民國50年4月2日", "PersonID": "B220757497", "spouse": "梁乙欽", "father": "易紹輝", "mother": "易廖菊鑾", "note": "原姓名:易美蓉。民國82年5月20日與梁乙欽結婚。原姓名易修禪名字不雅民國91年7月23日第二次改名。原住臺中縣烏日鄉九德村16鄰興祥街167巷22號七樓戶長本人民國93年8月26日遷入登記。原姓名易華茜係誤錄民國93年8月27日更正姓名。原姓梁民國93年8月27日撤冠夫姓登記。原姓名易華茜特殊原因民國106年5月5日第三次改名。"}, {"name": "梁錢茂"

2024-08-02 14:28:53,497 - INFO - Successfully processed file: 申請資料-4要件/戶籍/戶口7.pdf
2024-08-02 14:28:53,498 - INFO - Processing file 4/30: 申請資料-4要件/戶籍/戶籍3.pdf


...

2024-08-02 14:33:58,915 - INFO - Successfully processed file: 申請資料-4要件/戶籍/戶謄6.pdf
2024-08-02 14:33:58,916 - INFO - Processing file 30/30: 申請資料-4要件/戶籍/戶謄8.pdf


```jsonl
{"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶謄6.pdf", "IssueDate": "111/07/11", "IsFamilyRegisterExtract": True, "householdId": "FY552693", "householdAddress": "新北市新店區復興里015鄰中正路518巷15號四樓", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "黃玉琦", "title": "戶長", "birthDate": "民國55年7月20日", "PersonID": "F222578032", "spouse": "穆安迪(ANDY EFENDI)", "father": "黃李道榮", "mother": "黃甘妹", "note": "原住台北縣中和市民安里3鄰民安街25巷15號之3民國82年1月9日與周志能結婚不冠夫姓民國82年3月11日登記婚人。民國89年3月28日與周志能離婚民國89年3月28日申登。\n民國92年8月6日註記原住民民族別。原住臺北縣石碇鄉中民村9鄰十八重溪73號周志能戶內民國9\n2年8月6日遷入登記。民國92年12月17日與印尼國人穆安迪(ANDY EFENDI)(西元\n1973年6月1日出生)結婚民國93年2月25日申登。民國104年7月1日經法院裁判與印尼國\n人穆安迪(ANDY EFENDI)(1973年6月1日出生)離婚民國104年7月24日申登。"}, {"name": "周義挺", "title": "長子", "birthDate": "民國80年3月7日", "PersonID": "F128815149", "spouse": null, "father": "周志能", "mother": "黃玉琦", "note": "在台北市興隆路二段189號蔡婦產科出生民國82年5月17日申登。民國89年3月28\n日父母離婚約定由父監護民國89年3月28日申登。原住臺北縣石碇鄉中民村9鄰十八重溪73號周志\

2024-08-02 14:34:08,947 - INFO - Successfully processed file: 申請資料-4要件/戶籍/戶謄8.pdf
2024-08-02 14:34:08,953 - ERROR - Error decoding JSON: Expecting value: line 1 column 109 (char 108)


```jsonl
{"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶謄8.pdf", "IssueDate": "111/07/07", "IsFamilyRegisterExtract": True, "householdId": "F9277801", "householdAddress": "新北市永和區中溪里018鄰仁愛路186巷7號四樓", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "簡清熙", "title": "戶長", "birthDate": "民國47年1月1日", "PersonID": "S121955480", "spouse": "林明環", "father": "簡致", "mother": "劉牡丹", "note": "出生地臺灣省高雄縣。民國77年2月1日職變民國77年8月29日申登。民國78年5月1日職變民國78年5月16日申登。母簡劉牡丹民國89年11月8日撤冠姓民國89年12月2日申登。妻原姓名簡林明環民國103年10月31日撤冠配偶姓。"}, {"name": "林明環", "title": "妻", "birthDate": "民國50年4月15日", "PersonID": "F222105333", "spouse": "簡清熙", "father": "林萬鎰", "mother": "陳純", "note": "原住本市民族里10鄰民權路58巷1弄10號四樓戶長本人民國86年9月19日住變。原冠姓簡民國103年10月31日撤冠配偶姓登記。「"}, {"name": "簡川堯", "title": "長子", "birthDate": "民國72年5月21日", "PersonID": "S123205974", "spouse": null, "father": "簡清熙", "mother": "林明環", "note": "原住本市民族里10鄰民權路58巷1弄10號四樓簡林明環戶內民國86年9月19日住變。民國87年3月30日申報出生地。母

""


In [10]:
import json
import csv
import re


def preprocess_json_string(json_string):
    cleaned = json_string.replace('\n', '\\n')
    cleaned = cleaned.replace('True', 'true').replace('False', 'false')
    cleaned = re.sub(r'(\d+),\s*(\d+)', r'\1\2', cleaned)
    return cleaned


def parse_jsonl_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, 1):
            try:
                preprocessed_line = preprocess_json_string(line.strip())
                json_object = json.loads(preprocessed_line)
                data.append(json_object)
            except json.JSONDecodeError as e:
                print(f"第 {line_number} 行無法解析: {line.strip()}")
                print(f"錯誤信息: {str(e)}")
            except Exception as e:
                print(f"處理第 {line_number} 行時發生未知錯誤: {str(e)}")
    return data


def flatten_json(json_obj, parent_key='', sep='_'):
    items = []
    for k, v in json_obj.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_json(
                    {str(i): item}, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def json_to_csv(json_data, csv_file_path):
    flattened_data = [flatten_json(item) for item in json_data]

    # 獲取所有可能的列
    fieldnames = set()
    for item in flattened_data:
        fieldnames.update(item.keys())

    with open(csv_file_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=sorted(fieldnames))
        writer.writeheader()
        for item in flattened_data:
            writer.writerow(item)


# 使用示例
input_file_path = '戶籍combined_output.jsonl'
output_csv_file = '戶籍combined_output.csv'

parsed_data = parse_jsonl_file(input_file_path)
json_to_csv(parsed_data, output_csv_file)

print(f"成功解析的對象數量: {len(parsed_data)}")
print(f"CSV 文件已保存為: {output_csv_file}")

第 2 行無法解析: {"gsURL": "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶口5.pdf", "IssueDate": "111/07/15", "IsFamilyRegisterExtract": None, "householdId": "FOA2RMW5", "householdAddress": "新北市板橋區國泰里005鄰忠孝路忠義巷15弄96號三樓", "membersOfHousehold": {"description": "取得此戶籍地址紀錄的成員相關資訊並組合成一個嵌套的 JSON，提取其中所有成員的姓名、稱謂、出生日期、身份證字號、配偶、父親姓名、母親姓名、記事內容", "members": [{"name": "梁乙欽", "title": "戶長", "birthDate": "民國45年3月10日", "PersonID": "N120769673", "spouse": "易莉菲", "father": "梁秀森", "mother": "梁張摘", "note": "出生地:台灣省彰化縣。民國78年1月17日與梁江絲離婚。民國82年5月20日與易美蓉結婚。民國82年5月31日申登。妻梁易美蓉民國83年2月8日姓名變更民國93年8月27日申登。妻梁易修禪民國91年7月23日姓名變更民國93年8月27日申登。妻梁易華茜民國93年8月27日撤冠姓。原登記配偶姓名易華茜因配偶改名民國106年5月5日變更民國106年5月5日經彰化縣員林市戶政事務所逕為變更登記。"}, {"name": "易莉菲", "title": "妻", "birthDate": "民國50年4月2日", "PersonID": "B220757497", "spouse": "梁乙欽", "father": "易紹輝", "mother": "易廖菊鑾", "note": "原姓名:易美蓉。民國82年5月20日與梁乙欽結婚。原姓名易修禪名字不雅民國91年7月23日第二次改名。原住臺中縣烏日鄉九德村16鄰興祥街167巷22號七樓戶長本人民國93年8月26日遷入登記。原姓名易華茜係誤錄民國93年8月27日更正姓名。原姓梁民國93年8月27日撤冠夫姓登記。原姓名易華茜特殊原因民國106年5月5日第三次改名。"}, {"name": "梁錢

## 3. 財產查詢清單


#### 目前需要釐清的部分：

- 財產中只有土地需要被紀錄嗎？是需要被紀錄是否持有還是有其他資訊也需要讀取到？
- 財產中如果有房產資訊，是否需要紀錄房產的資訊？


In [28]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
  "IssuingAuthority": "提取此所得稅[核發單位]文字資訊放於此欄位",
  "AssetName": "提取[所得人姓名]",
  "AssetID": "提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼",
  "HasLand" : "如果財產列表中有土地相關的財產，請返回 True, 否則返回 False",
  "HasHouse" : "如果財產列表中有房屋相關的財產，請返回 True, 否則返回 False",
  "HasCar" : "如果財產列表中有汽車相關的財產，請返回 True, 否則返回 False",
  "HasAgriculturalLand": "如果財產列表中有田賦相關的財產，請返回 True, 否則返回 False",
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a Asset Inquiry List provided by the Taiwan government. You will extract the specified information from the PDF to JSONL and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

AssetInquiryList_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/財產/財產21.pdf"
AssetInquiryList_pdf_file = Part.from_uri(
    AssetInquiryList_pdf_file_uri, mime_type="application/pdf")
contents = [AssetInquiryList_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```jsonl
{"IssuingAuthority": "財政部北區國稅局中和稽徵所", "AssetName": "陳志祥", "AssetID": "U121541951", "HasLand": "True", "HasHouse": "True", "HasCar": "True", "HasAgriculturalLand": "True"}
{"IssuingAuthority": "財政部北區國稅局中和稽徵所", "AssetName": "黃郁喬", "AssetID": "A226350133", "HasLand": "False", "HasHouse": "False", "HasCar": "True", "HasAgriculturalLand": "False"}
{"IssuingAuthority": "財政部北區國稅局中和稽徵所", "AssetName": "陳念恩", "AssetID": "F234813608", "HasLand": "False", "HasHouse": "False", "HasCar": "False", "HasAgriculturalLand": "False"}
{"IssuingAuthority": "財政部北區國稅局中和稽徵所", "AssetName": "陳璽恩", "AssetID": "F232774957", "HasLand": "False", "HasHouse": "False", "HasCar": "False", "HasAgriculturalLand": "False"}
{"IssuingAuthority": "財政部北區國稅局中和稽徵所", "AssetName": "陳巧恩", "AssetID": "F233907683", "HasLand": "False", "HasHouse": "False", "HasCar": "False", "HasAgriculturalLand": "False"}
```


# 財產文檔正式處理

- 處理 30 個財產文檔，提取其中 Jsonl 資料並判斷是否有相關財產，並合併生成一個新的 Jsonl 檔案
- 再將 JsonL 資料轉換成 Dataframe
- 最後將 Dataframe 儲存成 CSV 檔案


In [33]:
def extract_json_strings(json_response):
    """
    提取JSON响应中的字符串函数。

    参数：
    - json_response：JSON响应字符串。

    返回值：
    - 字符串列表：提取的JSON响应中的字符串列表。

    说明：
    - 如果json_response为None，则返回空列表。
    - 移除json_response中的所有```json或```jsonl标记。
    - 使用正则表达式找出所有的JSON对象。
    - 解析每个JSON对象并重新序列化，以确保它们是有效的JSON。
    - 返回有效的JSON字符串列表。

    注意：
    - 忽略无效的JSON对象，继续处理下一个对象。
    """
    if json_response is None:
        return []

    # 移除所有的 ```json 或 ```jsonl 標記
    cleaned_response = re.sub(r'```(?:json|jsonl)?\s*', '', json_response)

    # 使用正則表達式找出所有的 JSON 對象
    json_objects = re.findall(r'\{[^{}]*\}', cleaned_response)

    # 解析每個 JSON 對象並重新序列化，以確保它們是有效的 JSON
    valid_json_strings = []
    for obj in json_objects:
        try:
            parsed = json.loads(obj)
            valid_json_strings.append(json.dumps(parsed, ensure_ascii=False))
        except json.JSONDecodeError:
            print(f"Invalid JSON: {obj}")
            continue  # 忽略無效的 JSON

    return valid_json_strings


def create_json_format(file_uri):
    return {
        "gsURL": file_uri,
        "IssuingAuthority": "提取此所得稅[核發單位]文字資訊放於此欄位",
        "AssetName": "提取[所得人姓名]",
        "AssetID": "提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼",
        "HasLand": "如果財產列表中有土地相關的財產，請返回 True, 否則返回 False",
        "HasHouse": "如果財產列表中有房屋相關的財產，請返回 True, 否則返回 False",
        "HasCar": "如果財產列表中有汽車相關的財產，請返回 True, 否則返回 False",
        "HasAgriculturalLand": "如果財產列表中有田賦相關的財產，請返回 True, 否則返回 False",
        "Date": "PDF 左下角的中華民國日期，格式為中華民國YYY年MM月DD日",
    }


def create_prompt(json_format):
    return f"""
    You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
    ```json format : {json.dumps(json_format, ensure_ascii=False, indent=4)}```
    Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.
    JSONL :
    """


file_prefix = "申請資料-4要件/財產/"
data_type = "財產"

df3 = process_all_documents(
    project_id, model_name, bucket_name, file_prefix, data_type)
print(df3.head())
print(f"Total rows in DataFrame: {len(df3)}")

2024-07-29 17:58:01,208 - INFO - Processing file 1/30: 申請資料-4要件/財產/財產1.pdf
2024-07-29 17:58:08,846 - INFO - Successfully processed file: 申請資料-4要件/財產/財產1.pdf
2024-07-29 17:58:08,848 - INFO - Processing file 2/30: 申請資料-4要件/財產/財產10.pdf
2024-07-29 17:58:15,343 - INFO - Successfully processed file: 申請資料-4要件/財產/財產10.pdf
2024-07-29 17:58:15,344 - INFO - Processing file 3/30: 申請資料-4要件/財產/財產11.pdf
2024-07-29 17:58:23,169 - INFO - Successfully processed file: 申請資料-4要件/財產/財產11.pdf
2024-07-29 17:58:23,170 - INFO - Processing file 4/30: 申請資料-4要件/財產/財產12.pdf
2024-07-29 17:58:31,037 - INFO - Successfully processed file: 申請資料-4要件/財產/財產12.pdf
2024-07-29 17:58:31,039 - INFO - Processing file 5/30: 申請資料-4要件/財產/財產13.pdf
2024-07-29 17:58:38,501 - INFO - Successfully processed file: 申請資料-4要件/財產/財產13.pdf
2024-07-29 17:58:38,502 - INFO - Processing file 6/30: 申請資料-4要件/財產/財產14.pdf
2024-07-29 17:58:45,695 - INFO - Successfully processed file: 申請資料-4要件/財產/財產14.pdf
2024-07-29 17:58:45,697 - INFO - Processing file

                                        gsURL IssuingAuthority AssetName  \
0   gs://nthurc-docai-poc/申請資料-4要件/財產/財產1.pdf     財政部北區國稅局板橋分局       邱冠憲   
1   gs://nthurc-docai-poc/申請資料-4要件/財產/財產1.pdf     財政部北區國稅局板橋分局      劉靜庭三   
2   gs://nthurc-docai-poc/申請資料-4要件/財產/財產1.pdf     財政部北區國稅局板橋分局       張堂齡   
3  gs://nthurc-docai-poc/申請資料-4要件/財產/財產10.pdf    財政部北區國稅局中和稽徵所       阮如冰   
4  gs://nthurc-docai-poc/申請資料-4要件/財產/財產10.pdf    財政部北區國稅局中和稽徵所       賈成龍   

      AssetID HasLand HasHouse HasCar HasAgriculturalLand            Date  
0  F126007881   False    False  False               False  中華民國111年06月30日  
1  L225864888   False    False  False               False  中華民國111年06月30日  
2  F224384387   False    False  False               False  中華民國111年06月30日  
3  F221898393   False    False  False               False  中華民國111年07月12日  
4  E120966700   False    False  False               False  中華民國111年07月11日  
Total rows in DataFrame: 113


In [38]:
df3.to_csv('財產測試資料.csv')

## 4. 社宅申請表

#### 需要討論的問題

- 手寫文件、是否要通過
- 是否只需要基本資訊 : 如申請人基本資料、申請項目等相關資訊
- 是否只需要他勾選了什麼的資訊即可，為勾選的則留空白


In [21]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
    "receiveDate" : 提取右上角[收件日期]欄位的手寫日期,
    "receiveNumber" : 提取右上角[收件編號]欄位的手寫編號,
    "applicantName": 提取表格上方中間申請人姓名的手寫內容，位於「本人」後面,
    "applicationDate": 提取表格中間右側[申請日期]欄位的手寫日期,
    "applicantBasics" : 一、申請人基本資料表{
        "ApplicantName": 提取[申請人姓名]欄位的手寫內容，位於表格左上角,
        "Gender": 提取性別欄位的勾選狀態，男/女，位於姓名欄位右側,
        "DateOfBirth": 提取[出生年月日]欄位的手寫內容，格式為民國年/月/日，位於性別欄位右側,
        "MaritalStatus": 提取婚姻欄位的勾選狀態，已婚/未婚，位於出生年月日欄位右側,
        "IDNumber": "提取[身分證統一編號]欄位的手寫內容，位於表格第二行左側",
        "HouseholdRegistrationNumber": "提取[戶口名簿戶號]欄位的手寫內容，位於表格第二行右側",
        "ContactPhone": "提取[聯絡電話]欄位的手寫內容，位於表格第三行左側",
        "MobilePhone": "提取[行動電話]欄位的手寫內容，位於表格第三行右側",
        "Email": "提取[電子郵件信箱]欄位的手寫內容，位於表格第四行",
        "RegisteredAddress": "提取[戶籍地址]欄位的手寫內容，包括郵遞區號、縣市、鄉鎮市區等信息，位於表格第五行",
        "MailingAddress": "提取[通訊地址]欄位的勾選狀態（同上）和可能的手寫內容，位於表格第六行",
        "EmergencyContact": "提取[緊急聯絡人]欄位的手寫內容，位於表格最後一行左側",
        "EmergencyContactPhone": "提取緊急聯絡人的[行動電話]欄位的手寫內容，位於表格最後一行右側"
        },
    "applicationItems": 二、申請項目表格{
        "ApplicantHousingType": "提取[申請房型]欄位的勾選狀態，位於表格左上方，包括一房/套房、二房、三房等選項",
        "ApplicantLocation": "提取[申請社區]欄位的勾選狀態(可複選，以列表呈現)，位於表格右上方，包括永和中正橋、板橋永翠等選項",
        "HouseholdType": "提取[一般戶]欄位下方的勾選狀態(可複選，以列表呈現)，位於表格中間偏左，包括一般青年戶、優先戶等選項",
        "PriorityGroup": "提取[優先戶]欄位下方的勾選狀態(可複選，以列表呈現)，位於表格下半部分，包括低收入戶、中低收入戶、特殊境遇家庭等選項",
        "DisabilityStatus": "提取[身心障礙者]欄位的勾選狀態，位於表格左下方",
        "SingleParentFamily": "提取[單親家庭]欄位的勾選狀態，位於表格左下方",
        "ViolenceVictim": "提取[家庭暴力或性侵害受害者及其子女]欄位的勾選狀態，位於表格左下方",
        "DisasterVictim": "提取[重大災害災民]欄位的勾選狀態，位於表格中下方",
        "ElderlyStatus": "提取[老人]欄位下方的勾選狀態，位於表格右下方，包括65歲以上、75歲以上等選項",
        "IndigenousStatus": "提取[原住民]欄位的勾選狀態，位於表格右下方",
        "OverseasChineseStatus": "提取[僑胞]欄位的勾選狀態，位於表格右下方",
        "EnvironmentallyDisplacedPerson": "提取[因環境災害或其他原因遷居者]欄位的勾選狀態，位於表格最下方",
    },
    "familyMembers":三、家庭成員資料{
        "familyMember": {
            "FamilyMemberName": 提取所有該家庭的成員，並順序輸出其姓名，並取得該人員詳細資料,
            "IDNumber": "提取[國民身分證統一編號/居留證統一證號]欄位的手寫內容，位於表格左側第二列",
            "Relationship": "提取[稱謂]欄位的手寫內容，位於表格右側第三列",
            "AnnualIncome": "提取[最近一年所得]欄位的手寫內容，位於表格最右側一列",
        },
        "TotalFamilyIncome": "提取[家戶最近一年總所得合計]欄位的手寫內容，位於表格底部倒數第二行",
        "AverageMonthlyIncome": "提取[平均每人每月所得]欄位的手寫內容，位於表格底部最後一行"
    }
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a Asset Inquiry List provided by the Taiwan government. You will extract the specified information from the PDF to JSONL and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

AssetInquiryList_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/申請書/申請書2.pdf"
AssetInquiryList_pdf_file = Part.from_uri(
    AssetInquiryList_pdf_file_uri, mime_type="application/pdf")
contents = [AssetInquiryList_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```json
{"receiveDate": "111.7.16", "receiveNumber": "YHC0100", "applicantName": "段榮昌", "applicationDate": "111年7月16日", "applicantBasics": {"ApplicantName": "段榮昌", "Gender": "男", "DateOfBirth": "70年12月7日", "MaritalStatus": "已婚", "IDNumber": "F125437447", "HouseholdRegistrationNumber": "FOA37QF4", "ContactPhone": "(02)22183748", "MobilePhone": "0905211535", "Email": "f7447kim0@yahoo.com.tw", "RegisteredAddress": "新北市(縣)中和區連坊里(村)18鄰連城路(街)段171巷弄30號7樓之", "MailingAddress": "同上", "EmergencyContact": "林雅甄", "EmergencyContactPhone": "0911590660"}, "applicationItems": {"ApplicantHousingType": "二房", "ApplicantLocation": ["板橋永翠"], "HouseholdType": ["一般青年戶"], "PriorityGroup": ["中低收入戶"], "DisabilityStatus": null, "SingleParentFamily": null, "ViolenceVictim": null, "DisasterVictim": null, "ElderlyStatus": null, "IndigenousStatus": null, "OverseasChineseStatus": null, "EnvironmentallyDisplacedPerson": null}, "familyMembers": {"familyMember": [{"FamilyMemberName": "段榮昌", "IDNumber": "F125437447", "Rel